In [1]:
import cv2 as cv
import mediapipe as mp

In [2]:
PATH = 'data/hand_shake.mp4'
video = cv.VideoCapture(PATH)

In [39]:
video = cv.VideoCapture(PATH)
hands = mp.solutions.hands.Hands()

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

while(video.isOpened()):
    ret, frame = video.read()
    
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    
    if ret == True:
        results = hands.process(frame)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                height, width, _ = frame.shape
                cx, cy = int(index_finger.x * width), int(index_finger.y * height)
                cv.circle(frame, (cx, cy), 5, (0, 0, 255), cv.FILLED)

                
        cv.imshow('frame', frame)
        
        if cv.waitKey(25) & 0xFF == ord('q'):
              break
    else: 
        break
video.release()
cv.destroyAllWindows()


In [82]:
def landmark_bbox(hand_landmarks, height, width) -> int :

    x_min, y_min = width, height
    x_max, y_max = 0, 0

    for landmark in hand_landmarks.landmark:
        x, y = int(landmark.x * width), int(landmark.y * height)

        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x)
        y_max = max(y_max, y)
    return x_min, y_min, x_max, y_max

In [81]:
def normalized_landmarks(hand_landmarks, bbox, height, width) -> float:
    x_min, y_min, x_max, y_max = bbox
    bbox_W, bbox_H = (x_max - x_min), (y_max - y_min)
    normalized_landmarks = []
    
    for landmark in hand_landmarks.landmark:
        x = (landmark.x * width - x_min) / bbox_W
        y = (landmark.y * height - y_min) / bbox_H
        normalized_landmarks.append((x, y))

    return normalized_landmarks

In [84]:
video = cv.VideoCapture(PATH)

hands = mp.solutions.hands.Hands()
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

Trail = []
while(video.isOpened()):
    ret, frame = video.read()
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    H, W, _ = frame.shape
    if ret == True:
        results = hands.process(frame)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                height, width, _ = frame.shape
                cx, cy = int(index_finger.x * width), int(index_finger.y * height)
                Trail.append((cx, cy))
                bbox = landmark_bbox(hand_landmarks, H, W)
                posotion = normalized_landmarks(hand_landmarks, bbox, H, W)
                print(posotion)
        for i in Trail:
            cv.circle(frame, i, 10, (0, 255, 0), cv.FILLED)
        cv.imshow('frame', frame)
        
        if cv.waitKey(25) & 0xFF == ord('q'):
              break
    else: 
        break
video.release()
cv.destroyAllWindows()

[(1.0070553000913847, 1.003142618551487), (0.7253976524423021, 0.9796891212463379), (0.47999957723355074, 0.873897564120409), (0.3273889384138475, 0.7605157480007265), (0.15999582273150803, 0.6836198248514315), (0.4812097811917646, 0.5986359293867902), (0.26954657878350774, 0.42155570518679736), (0.12324663914671731, 0.3176757126319699), (0.001106682173702695, 0.2336919540312232), (0.6298870821611597, 0.5326459989315127), (0.4636264766028168, 0.28315057987120096), (0.3457936278176964, 0.1276884369733857), (0.24756552320007885, 0.00442667123748035), (0.8008110676336726, 0.5163423549838182), (0.7036437288336798, 0.2750851119436869), (0.6306801192257383, 0.13021429282862965), (0.563858172215453, 0.012414158844366306), (0.9910146730755447, 0.5396733865505312), (0.9994553032271359, 0.35151760171099405), (0.9881239024870986, 0.2295898751514714), (0.9661302479035264, 0.12434784377493509)]
[(1.001252009950835, 1.0007580950216282), (0.7319343829977101, 0.9721999080634556), (0.4961683339086072, 